In [9]:
import pandas as pd
import glob
import os
import re
from tqdm import tqdm
from ekonlpy.sentiment import MPCK

In [21]:
!pip install ekonlpy

     ---------------------------------------- 0.0/33.2 MB ? eta -:--:--
     -- ------------------------------------- 2.1/33.2 MB 9.0 MB/s eta 0:00:04
     --------------- ----------------------- 12.8/33.2 MB 31.0 MB/s eta 0:00:01
     --------------------------- ----------- 23.1/33.2 MB 36.5 MB/s eta 0:00:01
     --------------------------------------  32.8/33.2 MB 38.5 MB/s eta 0:00:01
     ---------------------------------------- 33.2/33.2 MB 36.9 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/10.3 MB ? eta -:--:--
   -------------------------------------- - 10.0/10.3 MB 47.8 MB/s eta 0:00:01
   ---------------------------------------- 10.3/10.

  You can safely remove it manually.
  You can safely remove it manually.


In [ ]:

def clean_news(text):
    if not isinstance(text, str):
        return ''
    
    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    
    # 기자 이메일, 언론사 이름 제거
    text = re.sub(r'\[.+?기자\]|\[.+?뉴스\]|\[.+?제공\]', '', text)
    text = re.sub(r'\/사진제공?=.+?$', '', text)
    text = re.sub(r'\/사진?=.+?$', '', text)
    
    # 홍보 문구 제거
    text = re.sub(r'이\s*기사는\s*.+?\s*에\s*게재된\s*기사입니다\.', '', text)

    # 줄바꿈 공백으로
    text = text.replace('\u2028', ' ').replace('\u2029', ' ').replace('\n', ' ')
    
    # 한글, 영어, 숫자, 공백, 마침표만 남기기 (마침표는 문장 구분을 위해 남겨야 함)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s\.]', ' ', text)
    
    # 연속된 공백 하나로 줄이기
    text = ' '.join(text.split())
    
    return text

In [ ]:
# 여러 언론사 클렌징 데이터 합치기
tqdm.pandas()

folder_path = '../../db/news_contents/news_contents_*.csv'
file_list = glob.glob(folder_path)

all_data = []

for file in file_list:
    filename = os.path.basename(file)
    source_name = filename.split('_')[-1].replace('.csv', '')

    df = pd.read_csv(file, usecols=['date', 'content'])

    # 오염 데이터 제거용
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.dropna(subset=['date'])

    temp_df = pd.DataFrame()
    temp_df['date'] = pd.to_datetime(df['date'])
    temp_df['date'] = temp_df['date'].dt.date

    temp_df['content'] = df['content'].progress_apply(clean_news)
    temp_df['tokens'] = ''
    temp_df['category'] = '뉴스'
    temp_df['source'] = source_name

    all_data.append(temp_df)
    print(f"✔ {source_name} 처리 완료 ({len(temp_df)}건)")

if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    final_df = final_df[['date', 'content', 'tokens', 'category', 'source']]
    final_df.to_csv('news_preprocessed_integrated.csv', index=False, encoding='utf-8-sig')
    print(f"\n✨ 총 {len(final_df)}건의 '뉴스' 데이터 저장 완료")


100%|██████████| 71236/71236 [00:03<00:00, 21872.43it/s]


✔ 매일경제 처리 완료 (71236건)


100%|██████████| 89012/89012 [00:05<00:00, 17015.67it/s]


✔ 머니투데이 처리 완료 (89012건)


100%|██████████| 79208/79208 [00:04<00:00, 19770.41it/s]


✔ 한국경제 처리 완료 (79208건)

✨ 총 239456건의 '뉴스' 데이터 저장 완료


In [ ]:

def ngramize(tokens, max_n=5):
    keep_tags = ['NNG', 'VV', 'VA', 'MAG']
    filtered = [w for w in tokens if w.split('/')[1] in keep_tags]
    ngram_results = []
    for pos in range(len(filtered)):
        for n in range(1, max_n + 1):
            if pos + n <= len(filtered):
                ngram = ";".join(filtered[pos : pos + n])
                ngram_results.append(ngram)
                
    return ngram_results

mpck = MPCK()

def get_tokens(text):
    try:
        if not text or len(str(text)) < 10:
            return ""
        tokens = mpck.tokenize(text)
        ngrams = ngramize(tokens)
        return ", ".join(tokens+ngrams)
    except:
        return ""

In [ ]:
input_file = 'news_preprocessed_integrated.csv'
output_file = 'news_preprocessed_output.csv'
batch_size = 5000 

if os.path.exists(output_file):
    print(f"♻️ 기존에 작업하던 '{output_file}'을 발견했습니다. 이어서 시작합니다.")
    df = pd.read_csv(output_file)
else:
    print(f"🆕 처음부터 시작합니다. '{input_file}'을 불러옵니다.")
    df = pd.read_csv(input_file)
    if 'tokens' not in df.columns:
        df['tokens'] = ""


to_process = df[df['tokens'].isna() | (df['tokens'] == "")].index
print(f"🚀 남은 작업량: {len(to_process)}건 / 전체: {len(df)}건")

try:
    for i, idx in enumerate(tqdm(to_process)):
        df.at[idx, 'tokens'] = get_tokens(df.at[idx, 'content'])
        
        if (i + 1) % batch_size == 0 or (i + 1) == len(to_process):
            df.to_csv(output_file, index=False, encoding='utf-8-sig')
            
except KeyboardInterrupt:
    print("\n🛑 사용자가 작업을 중단했습니다. 현재까지의 내용은 안전하게 저장되었습니다.")

print(f"✨ 모든 작업 완료! 결과 파일: {output_file}") 

♻️ 기존에 작업하던 'news_preprocessed_output.csv'을 발견했습니다. 이어서 시작합니다.
🚀 남은 작업량: 3836건 / 전체: 239456건


100%|██████████| 3836/3836 [00:16<00:00, 238.26it/s] 

✨ 모든 작업 완료! 결과 파일: news_preprocessed_output.csv


In [ ]:
import pandas as pd
import json
import csv

df_news = pd.read_csv('news_preprocessed_output.csv', encoding='utf-8-sig')

def string_to_list(text):
    if pd.isna(text): return "[]" # 빈 칸 처리
    clean_list = [t.strip() for t in str(text).split(',') if t.strip()]
    return json.dumps(clean_list, ensure_ascii=False)

df_news['tokens'] = df_news['tokens'].apply(string_to_list)

output_path = 'news_preprocessed_fixed.csv'
df_news.to_csv(output_path, index=False, encoding='utf-8-sig', quoting=csv.QUOTE_ALL)

print(f"✨ 기사 데이터 수정 완료! 이제 리스트 형식입니다: {output_path}")

✨ 기사 데이터 수정 완료! 이제 리스트 형식입니다: news_data_fixed.csv


In [12]:
df_news.head(10)

,date,content,tokens,category,source
0,2025-11-14,14일 서울 중구 하나은행 본점 딜링룸 현황판에 코스피가 표시돼 있다. 나흘 연속 ...,"[""중구/NNG"", ""본점/NNG"", ""딜링룸/NNG"", ""현황/NNG"", ""판/N...",뉴스,매일경제
1,2025-10-21,NH농협은행이 수탁 중인 전국 광역자치단체 금고의 예치액에 대한 금리가 최대 1.2...,"[""수탁/NNG"", ""중/NNG"", ""전국/NNG"", ""금고/NNG"", ""예치/NN...",뉴스,매일경제
2,2025-12-18,유럽중앙은행 ECB 이 18일 현지시간 예금금리를 비롯한 3대 정책금리를 모두 동결...,"[""ecb/NNG"", ""ecb/NNG"", ""현지/NNG"", ""예금/NNG"", ""금리...",뉴스,매일경제
3,2025-12-14,은행채 금리상승에 조달비용 대형증권사 IMA 출시도 변수 증권사 머니무브 차단 목적...,"[""은행채/NNG"", ""금리/NNG"", ""상승/NNG"", ""조달/NNG"", ""비용/...",뉴스,매일경제
4,2025-11-22,비트코인 급락에 스트레티지도 60 폭락 MSCI 나스닥100등 벤치마크 제외가능성 ...,"[""비트코인/NNG"", ""급락/NNG"", ""하락/NNG"", ""msci/NNG"", ""...",뉴스,매일경제
5,2025-12-30,AI 시대 자본도 빠르게 이동 고환율 부동산 은퇴 자산 주목 R E D H O R ...,"[""시대/NNG"", ""자본/NNG"", ""빠르/VA"", ""이동/NNG"", ""고환/NN...",뉴스,매일경제
6,2025-11-16,저원가성 예금 확보 위한 수단 브랜드 재정비하고 혜택 확대 4대 시중은행이 브랜드를...,"[""원가/NNG"", ""예금/NNG"", ""확보/NNG"", ""위하/VV"", ""브랜드/N...",뉴스,매일경제
7,2025-11-07,내년 3기 사업 시작 앞두고 신규서비스 디자인 경쟁 신한 하나 IBK기업은행이 나라...,"[""사업/NNG"", ""시작/NNG"", ""앞두/VV"", ""신규/NNG"", ""서비스/N...",뉴스,매일경제
8,2025-12-30,외환당국 성탄전야 환율단속 1992년 블랙웬즈데이엔 중앙은행도 펀드에 굴복 성장정책...,"[""외환당국/NNG"", ""탄전/NNG"", ""환율/NNG"", ""단속/NNG"", ""블랙...",뉴스,매일경제
9,2012-01-26,매경테스트 예제 금리에 관한 다음의 설명 중 올바른 것은. 단리는 복리보다 많은 이...,"[""테스트/NNG"", ""예제/NNG"", ""금리/NNG"", ""관하/VV"", ""설명/N...",뉴스,매일경제
